# Class II - An introduction to JuMP

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of JuMP. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) first.

**Warning! This notebook is an introduction to JuMP v0.18. However, JuMP is currently undergoing a re-write. In the near future, some aspects of JuMP will change. But don't worry, the majority of this tutorial is still relevant in the future.**

This tutorial doesn't exist in isolation. Some other good resources for learning JuMP are
- [the Discourse forum](https://discourse.julialang.org/c/domain/opt)
- [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18/)
- [JuMP examples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.18/examples)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia/)

As in Class I, run the following magic sauce to check we're good to go.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

## The basics

In [33]:
model = Model()
@variable(model, x)
@variable(model, y >= 0)
@variable(model, 1 <= z <= 2)
model

Feasibility problem with:
 * 0 linear constraints
 * 3 variables
Solver is default solver

We can also create arrays of JuMP variables.

In [31]:
model = Model()
@variable(model, x[i = 1:4] >= i)
x

4-element Array{Variable,1}:
 x[1]
 x[2]
 x[3]
 x[4]

The indices of the arrays don't have to be integers. They can be anything, like a string or a symbol.

In [29]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)
x

x[i,j] >= .. for all i in {1,2}, j in {A,B}

We can also add a condition to filter out some of the variables:

In [35]:
model = Model()
@variable(model, x[i = 1:4, j = [:A, :B]; isodd(i)] >= i)
x

x[i,j] >= .. for all i in {1,2,3,4}, j in {A,B} s.t. isodd(i)

What if I want to add two variables with the same name?

In [41]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

┌ Warning: A variable or constraint named x is already attached to this model. If creating variables programmatically, use the anonymous variable syntax x = @variable(m, [1:N], ...).
└ @ JuMP C:\Users\Oscar\.julia\packages\JuMP\Xvn0n\src\JuMP.jl:852


x

### Quiz Question

What is the value of the following?

In [ ]:
JuMP.getlowerbound(x)

In [34]:
@variable(model, x)

# Gets expanded to something like:
x = @variable(model, basename = "x")
model[:x] = x


x

Anonymous variables

## Example: Economic dispatch

*This example is adapted from a [tutorial given at the 2015 Winter School](https://github.com/JuliaOpt/juliaopt-notebooks/blob/3110eddaf5effdecfee687739295bea05731ba33/notebooks/Dvorkin%20-%20Power%20systems%20-%20Economic%20dispatch%20and%20Unit%20commitment.ipynb).*

Economic dispatch (ED) is an optimization problem that minimizes the cost of supplying energy demand subject to operational constraints on power system assets. In its simplest modification, ED is an LP problem solved for an aggregated load and wind forecast and for a single infinitesimal moment. Mathematically, the ED problem can be written as follows:
$$
\min \sum_{i \in I} c^g_{i} \cdot g_{i} + c^w \cdot w,
$$
where $c_{i}$ and $g_{i}$ are the incremental cost ($\$/MWh$) and power output ($MW$) of the $i^{th}$ thermal generator, respectively, and $c^w$ and $w$ are the incremental cost ($\$/MWh$) and wind power injection ($MW$), respectively.

s.t.

<li> Minimum ($g^{\min}$) and maximum ($g^{\max}$) limits on power outputs of the thermal generators: </li>
$$
g^{\min}_{i} \leq g_{i} \leq g^{\max}_{i}.
$$
<li>Constraint on the wind power injection:</li>
$$
0 \leq w \leq w^f, 
$$
where $w$ and $w^f$ are the wind power injection and wind power forecast, respectively.

<li>Power balance constraint:</li>
$$
\sum_{i \in I} g_{i} + w = d, 
$$
where $d$ is the demand.

Further reading on ED models can be found in A. J. Wood, B. F. Wollenberg, and G. B. Sheblé, "Power Generation, Operation and Control", Wiley, 2013.

### JuMP Implementation of Economic Dispatch 

First, we need to load some packages.

In [9]:
using JuMP
using GLPKMathProgInterface
using Interact
using Plots

Now, we define some problem data.

In [57]:
# Define an instance of the GLPK solver.
const GLPK_SOLVER = GLPKMathProgInterface.GLPKSolverLP()

# Define some input data about the test system
# Maximum power output of generators
const GENERATION_MAX = [1000,1000]
# Minimum power output of generators
const GENERATION_MIN = [0,300]
# Incremental cost of generators 
const COST_GENERATION = [50,100]
# Incremental cost of wind generators
const COST_WIND = 50
# Total demand
const DEMAND = 1500
# Wind forecast
const WIND_MAX = 200;

In the next cell, we create a Julia function that formulates and solves the economic dispatch problem.

In [ ]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)
    economic_dispatch = Model(solver = GLPK_SOLVER) 
    
    # Define decision variables    
    @variables(economic_dispatch, begin
        g[i=1:2]  # Thermal generation (MW).
        w >= 0  # Wind power (MW).
    end)

    # Define the objective function
    @objective(economic_dispatch, Min, 
        sum(cost_of_thermal[i] * g[i] for i in 1:2) + cost_of_wind * w
    )

    # Define the constraint on the maximum and minimum power output of each generator.
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end
    
    @constraints(economic_dispatch, begin
        # Define the constraint on the wind power injection
        w <= WIND_MAX
        # Define the power balance constraint
        sum(g[i] for i in 1:2) + w == DEMAND
    end)

    # Solve statement
    solve(economic_dispatch)
    
    # Return the optimal value of the objective function and its minimizers
    # as a NamedTuple.
    return (
        generation = getvalue(g), 
        wind_generation = getvalue(w),
        wind_spillage = w_f - getvalue(w),
        cost = getobjectivevalue(economic_dispatch)
    )
end

# Solve the economic dispatch problem
solution = solve_economic_dispatch()

println("Dispatch")
println("    Generators: ", solution.generation[1:2], " MW")
println("    Wind: ", solution.wind_generation, " MW")
println("Wind spillage: ", solution.wind_spillage, " MW") 
println("----------------------------------")
println("Total cost: \$", solution.cost)  

### Economic dispatch with adjustable incremental costs

In the following exercise we introduce a manipulator to vary the cost of wind generation and observe its impact the total cost, dispatch of generators G1 and G2, utilization of available wind under different values of the incremental cost of generator G1.

In [ ]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.1:3.5)
    # Define the vectors of outputs
    objectives = Float64[] 
    wind_dispatch = Float64[]
    G1_dispatch = Float64[]
    G2_dispatch = Float64[]
    
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.01:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        # Add the solution of the economic dispatch problem to the respective vectors
        push!(objectives, solution.cost)
        push!(wind_dispatch, solution.wind_generation)
        push!(G1_dispatch, solution.generation[1])
        push!(G2_dispatch, solution.generation[2])
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, objectives,
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, G1_dispatch,
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, G2_dispatch,
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, wind_dispatch,
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

## Nonlinear example

JuMP can also be used to solve non-linear problems.

In [ ]:
solver = IpoptSolver()

## Composing models

In [9]:
"""
    my_abs(x::JuMP.Variable)

Return a variable that takes the value of the absolute
value (L1-norm) of `x`.
"""
function my_abs(x::JuMP.Variable)
    model = x.m
    x⁺ = @variable(model, lowerbound = 0)
    x⁻ = @variable(model, lowerbound = 0)
    abs_x = @variable(model, lowerbound = 0)
    @constraints(model, begin
        x⁺ >= x
        x⁻ >= -x
        abs_x == x⁺ + x⁻
    end)
    return abs_x
end

model = Model(solver = GLPKSolverLP())
@variable(model, x)
@objective(model, Min, my_abs(x))
JuMP.fix(x, -1)
JuMP.solve(model)
println("|x| = |$(JuMP.getvalue(x))| = $(JuMP.getobjectivevalue(model))")

JuMP.fix(x, 2)
JuMP.solve(model)
println("|x| = |$(JuMP.getvalue(x))| = $(JuMP.getobjectivevalue(model))")

|x| = |-1.0| = 1.0
|x| = |2.0| = 2.0
